In [14]:
def riscv(instruction):
    op,i0,i1,output = instruction ##assigns the instruction components
    
    arithop = {'Plus':'ADD','Minus':'SUB','Times':'MUL'}
    boolop = {'Greater':'BGT','GreaterEq':'BGE','Equal':'BEQ','And':'AND','Or':'OR'}
    jumpop = {'JumpIfZero':'BEQZ','JumpIfNonzero':'BNEZ'}
    opname = arithop | boolop | jumpop ##joins the above dictionaries
    
    name = opname[op] ##gets the riscv name of the instruction
    
    if op in arithop:
        riscv = "LW " + i0 + " a0" + '\n' \
                + "LW " + i1 + " a1" + '\n' \
                + name + " " + "a1 a1 a0" + '\n' \
                + "SW a1 " + output + '\n'
    
    elif op in boolop: 
        riscv = "LW " + i0 + " a0" + '\n' \
                + "LW " + i1 + " a1" + '\n' \
                + name + " " + "a0 a1 " + output + '\n'
    
    elif op in jumpop:
        riscv = "LW " + i0 + " a0" + '\n' \
                + name + " " + "a0 " + i1 + '\n'
    
    return riscv
    
with open('../example_op_file.txt', 'r') as f:
    operations = f.readlines()
    instructions = [o.split(' ') for o in operations]
#     for operation in operations:
#         instruction = operation.split(' ')
#         instructions.append(instruction)
    for inst in instructions:
        print(inst)
        print(riscv(inst))

['Plus', '1', '2', '3\n']
LW 1 a0
LW 2 a1
ADD a1 a1 a0
SW a1 3


['Minus', '1', '2', '3\n']
LW 1 a0
LW 2 a1
SUB a1 a1 a0
SW a1 3


['Times', '1', '2', '3\n']
LW 1 a0
LW 2 a1
MUL a1 a1 a0
SW a1 3


['Greater', '1', '2', '3\n']
LW 1 a0
LW 2 a1
BGT a0 a1 3


['GreaterEq', '1', '2', '3\n']
LW 1 a0
LW 2 a1
BGE a0 a1 3


['Equal', '1', '2', '3\n']
LW 1 a0
LW 2 a1
BEQ a0 a1 3


['Or', '1', '2', '3\n']
LW 1 a0
LW 2 a1
OR a0 a1 3


['And', '1', '2', '3\n']
LW 1 a0
LW 2 a1
AND a0 a1 3


['Not', '1', 'null', '3\n']


KeyError: 'Not'